In [1]:
from qdrant_client import QdrantClient, models
from FlagEmbedding import BGEM3FlagModel
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from utils.utils import convert_defaultdict, format_docs

from dotenv import load_dotenv
load_dotenv()

/home/taiquach123471/anaconda3/envs/chatbots/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
client = QdrantClient("http://localhost:6333")
embeddings = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
llm = ChatGroq(model="llama3-70b-8192")

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 58470.78it/s]
/home/taiquach123471/anaconda3/envs/chatbots/lib/python3.11/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub 

In [9]:
system = """You are an AI chatbot designed to answer questions about insomnia using accurate information from scientific articles. Your goal is to provide clear, evidence-based responses and practical advice to help users manage their insomnia. Following those instructions:
1. Context: Use only the information provided with the <context> tags regarding insomnia.
2. Answer Format: Provide concise, informative answers, citing relevant research where applicable.
3. User Interaction: If a question is unclear or needs more context, ask the user for clarification."""

human = """User query: {query}

<context>
{context}
</context>
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", human)
])

chain = prompt | llm | StrOutputParser()

In [12]:
def retrieve(query, embeddings, client):
    res = embeddings.encode([query], return_sparse=True, return_colbert_vecs=True)
    result = client.query_points(
        "semantic_vectorstore",
        prefetch=[
            models.Prefetch(
                query=res['dense_vecs'][0],
                using="dense",
                limit=20
            ),
            models.Prefetch(
                query=models.SparseVector(**convert_defaultdict(res['lexical_weights'][0])),
                using="sparse",
                limit=20
            ),
            models.Prefetch(
                query=res['colbert_vecs'][0],
                using='colbert',
                limit=20
            )
        ],
        query=models.FusionQuery(
            fusion=models.Fusion.RRF,
        ),
        limit=3
    )

    
    return result

In [13]:
query = "I'm 22 years old and I usually sleep at 2am. How can this affect my health?"

In [15]:
result = retrieve(query=query, embeddings=embeddings, client=client)

ResponseHandlingException: timed out